In [1]:
import Pkg
Pkg.activate(".")

  Activating project at `~/Documents/programming/BDTools.jl/examples`


In [2]:
Pkg.add(["Plots",
        "Lux",
        "ADTypes",
        "MLUtils","Zygote","HDF5",
        "Optimization","OptimizationOptimJL","OptimizationOptimisers",
        "SciMLSensitivity", "ComponentArrays"])

   Resolving package versions...
  No Changes to `~/Documents/programming/BDTools.jl/examples/Project.toml`
  No Changes to `~/Documents/programming/BDTools.jl/examples/Manifest.toml`


In [3]:
using HDF5: HDF5
using Plots, Random, Lux, ADTypes, Zygote, Printf, MLUtils, Statistics
using Optimization, Optimisers, OptimizationOptimisers, SciMLSensitivity, ComponentArrays

In [4]:
struct GroundTruthCleaned
    data::Array{Float64,3}
end
    
function deserialize(filepath::String, ::Type{GroundTruthCleaned})
    HDF5.h5open(filepath, "r") do io
        GroundTruthCleaned(
            io["GroundTruthCleaned/data"] |> read
        )
    end
end

deserialize (generic function with 1 method)

In [5]:
"""
    standardize(data::AbstractArray; dims=1)

Return standardized `data` over dimension `dims`.
"""
function standardize(data::AbstractArray; dims=1)
    μ = mean(data, dims=dims)
    σ = std(data, dims=dims, mean=μ)
    (data.-μ) ./ σ, μ, σ
end

standardize

In [6]:
gt = deserialize("gt_clean.h5", GroundTruthCleaned)
ori64, ori_mean, ori_std = standardize(reshape(gt.data[:,:,1],
    (size(gt.data[:,:,1])[1],1,size(gt.data[:,:,1])[2])))
sim64, sim_mean, sim_std = standardize(reshape(gt.data[:,:,2],
    (size(gt.data[:,:,2])[1],1,size(gt.data[:,:,2])[2])))
ori = Float32.(ori64)
sim = Float32.(sim64)

400×1×1056 Array{Float32, 3}:
[:, :, 1] =
 -0.25204962
  0.8046118
 -1.1035193
 -0.79414064
  1.9682941
  0.5637778
 -0.012230185
 -0.21458794
 -0.07579102
  1.606328
 -0.5129932
 -0.49603918
  1.8115286
  ⋮
  0.60853773
 -0.38985053
 -0.50285214
 -0.4290375
 -0.2008933
  0.90451586
 -0.26445413
  0.938424
 -1.2621317
  1.3216174
  1.6147438
  1.8672619

[:, :, 2] =
  0.46098885
  0.7983072
  0.9000875
 -0.5965039
  1.0699972
 -0.62849957
 -1.6455749
 -0.9128678
 -0.49982587
  1.4695151
 -1.2132338
  1.2019722
 -0.807777
  ⋮
  0.23529959
  0.5053252
 -0.57064825
 -0.05665371
  0.18813378
 -1.2495686
  0.4842252
 -0.3646048
  0.07366619
  2.0850706
  0.35293218
  0.8921558

[:, :, 3] =
  0.2637831
 -0.953591
  1.8452263
  0.82109296
  0.5363555
 -0.87426835
 -0.23479255
 -2.2671652
  0.2375749
  0.2216988
 -0.96207213
 -1.3896456
  0.358906
  ⋮
 -0.8054758
 -0.74574023
  0.6919415
 -0.61785483
 -1.5915744
  1.1232259
  0.41837677
  0.8646118
 -0.68044275
  0.41274798
 -1.5104008
 -0.963

In [7]:
cor(vec(ori),vec(sim))

0.24442405f0

In [8]:
(i_train, o_train), (i_test, o_test) = splitobs((ori, sim); at=0.8)
train_dataloader = DataLoader(collect.((i_train, o_train)); batchsize=8, shuffle=true)
test_dataloader = DataLoader(collect.((i_test, o_test)); batchsize=8, shuffle=true)

27-element DataLoader(::Tuple{Array{Float32, 3}, Array{Float32, 3}}, shuffle=true, batchsize=8)
  with first element:
  (400×1×8 Array{Float32, 3}, 400×1×8 Array{Float32, 3},)

In [9]:
rng = MersenneTwister()
Random.seed!(rng, 12345)

MersenneTwister(12345)

In [10]:
function negr2(y_pred, y_true)
    SS_tot = sum(abs2, vec(y_true) .- mean(vec(y_true)))
    return negr2(y_pred, y_true, SS_tot)
end

function negr2(y_pred, y_true, SS_tot)
    SS_res =  sum(abs2, vec(y_true) .- vec(y_pred))
    r2 = 1 - SS_res/(SS_tot + eps())
    return -r2
end

negr2 (generic function with 2 methods)

In [11]:
model = Chain(
        Conv((9,), 1=>18, sigmoid, pad=SamePad()),
        [Chain(Conv((9,), 18=>18, pad=SamePad()), BatchNorm(18, sigmoid)) for _ in 1:6]...,
        Dropout(0.2),
        Conv((1,), 18=>1, pad=SamePad())
    )

Chain(
    layer_1 = Conv((9,), 1 => 18, σ, pad=4),  # 180 parameters
    layer_2 = Chain(
        layer_1 = Conv((9,), 18 => 18, pad=4),  # 2_934 parameters
        layer_2 = BatchNorm(18, σ, affine=true, track_stats=true),  # 36 parameters, plus 37
    ),
    layer_3 = Chain(
        layer_1 = Conv((9,), 18 => 18, pad=4),  # 2_934 parameters
        layer_2 = BatchNorm(18, σ, affine=true, track_stats=true),  # 36 parameters, plus 37
    ),
    layer_4 = Chain(
        layer_1 = Conv((9,), 18 => 18, pad=4),  # 2_934 parameters
        layer_2 = BatchNorm(18, σ, affine=true, track_stats=true),  # 36 parameters, plus 37
    ),
    layer_5 = Chain(
        layer_1 = Conv((9,), 18 => 18, pad=4),  # 2_934 parameters
        layer_2 = BatchNorm(18, σ, affine=true, track_stats=true),  # 36 parameters, plus 37
    ),
    layer_6 = Chain(
        layer_1 = Conv((9,), 18 => 18, pad=4),  # 2_934 parameters
        layer_2 = BatchNorm(18, σ, affine=true, track_stats=true),  # 36 parameters, plus 

In [12]:
ps, st = Lux.setup(rng, model)
ps_ca = ComponentArray(ps)
nothing

In [13]:
function loss(p, (i_data,o_data))
    model_d = Lux.apply(model,i_data,p, st)[1]
#    print(size(i_data))
#    return negr2(vec(model_d),vec(o_data))
    c = cor(vec(model_d),vec(o_data))
    return -c/(1-c) # stretch the range to -infty to +infty
end

loss (generic function with 1 method)

In [14]:
losses = []
corrs = []
bestpara = nothing
bestc = 0.0

function callback(state, l)
    model_d = Lux.apply(model,i_test,state.u, st)[1]
    c = cor(vec(model_d),vec(o_test))
    if c > bestc
        global bestc = c
        global bestpara = state.u
    end
    push!(losses,l)
    push!(corrs,c)
    state.iter % 50 == 1 && @printf "Iteration: %5d, Loss: %.5g, Fidelity: %.5g\n" state.iter l c
    return false
end

function callback2(state, l)
    model_d = Lux.apply(model,i_test,state.u, st)[1]
    c = cor(vec(model_d),vec(o_test))
    if c > bestc
        global bestc = c
        global bestpara = state.u
    end
    push!(losses,l)
    push!(corrs,c)
    @printf "Iteration: %5d, Loss: %.5g, Fidelity: %.5g\n" state.iter l c
    return false
end

callback2 (generic function with 1 method)

In [15]:
opt_func = OptimizationFunction(loss, Optimization.AutoZygote())
opt_prob = OptimizationProblem(opt_func, ps_ca, test_dataloader)
opt = Adam(1e-4)

Adam(0.0001, (0.9, 0.999), 1.0e-8)

In [16]:
epochs = 10
res_adam = solve(opt_prob, opt; callback, maxiters = epochs)

LoadError: UndefVarError: `x` not defined in local scope
Suggestion: check for an assignment to a local variable that shadows a global of the same name.

In [17]:
cor(vec(i_test),vec(o_test)), cor(vec(i_train),vec(o_train))

(0.22671476f0, 0.24884558f0)

In [18]:
cor(vec(Lux.apply(model,i_test,bestpara, st)[1]),vec(o_test)), cor(vec(Lux.apply(model,i_train,bestpara, st)[1]),vec(o_train))

LoadError: type Nothing has no field layer_1

In [19]:
cor(vec(Lux.apply(model,i_test,res_adam.u, st)[1]),vec(o_test)), cor(vec(Lux.apply(model,i_train,res_adam.u, st)[1]),vec(o_train))

LoadError: UndefVarError: `res_adam` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
epochs_ = 50
opt_prob_new = OptimizationProblem(opt_func, res_adam.u, (i_train,o_train))

In [ ]:
res_bfgs = solve(opt_prob_new, LBFGS(); callback=callback2, maxiters = 500)